In [1]:
import tensorflow as tf

import os
import numpy as np
#os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
#os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import pandas as pd

import QuantSimul

In [2]:
BATCH_SIZE = 128
ACT_QUANT_START = 500000
ACT_QUANT_START = round(ACT_QUANT_START/BATCH_SIZE)

In [3]:
def BasicBlock_QuantSimul(x, channel_size, name, strides=1, kernel_init="glorot_uniform", kernel_reg = None):
    x_1 = x
    x_2 = x
    
    if strides != 1:
        #option_a
        x_2 = tf.keras.layers.MaxPool2D(1, strides=strides, padding="same", name=name+"_sc_maxpool"  )( tf.pad(x_2, ((0,0), (0,0), (0,0), (0, channel_size-x_2.shape[-1])), name=name+"_sc_optionA") )
        #option_b
        #x_2 = tf.keras.layers.Conv2D(channel_size, 1, strides=strides, padding='same', use_bias=False, name=name+"_sc_conv", kernel_initializer=INIT)(x_2)
        #x_2 = tf.keras.layers.BatchNormalization(name=name+"_sc_bn")(x_2)

    x_1 = QuantSimul.QuantSimulConv2D(channel_size, 3, strides=strides, padding="same", use_bias=False, name=name+"_conv1", kernel_initializer=kernel_init, kernel_regularizer=kernel_reg)(x_1)
    x_1 = tf.keras.layers.BatchNormalization(name=name+"_bn1")(x_1)
    x_1 = tf.keras.layers.Activation('relu6', name=name+"_act1")(x_1)
    x_1 = QuantSimul.QuantActivation(qunatization_start=ACT_QUANT_START, name=name+"_act1_quantized")(x_1)
    
    x_1 = QuantSimul.QuantSimulConv2D(channel_size, 3, strides=1, padding="same", use_bias=False, name=name+"_conv2", kernel_initializer=kernel_init, kernel_regularizer=kernel_reg)(x_1)
    x_1 = tf.keras.layers.BatchNormalization(name=name+"_bn2")(x_1)
    
    x = tf.keras.layers.Add(name=name+"_add")([x_1, x_2])
    x = tf.keras.layers.Activation('relu6', name=name+"_act2")(x)
    x = QuantSimul.QuantActivation(qunatization_start=ACT_QUANT_START, name=name+"_act2_quantized")(x)

    return x

def ResNet20_QuantSimul(input_shape=(32,32,3), classes=10, channel_sizes=16, kernel_init="glorot_uniform", kernel_reg=tf.keras.regularizers.L2(1e-4)):
    inputs = tf.keras.Input(shape=input_shape)
    x = inputs
    
    #pre
    x = QuantSimul.QuantSimulConv2D(channel_sizes, 3, strides=1, padding="same", use_bias=False, name="pre_conv", kernel_initializer=kernel_init, kernel_regularizer=kernel_reg)(x)
    x = tf.keras.layers.BatchNormalization(name="pre_bn")(x)
    x = tf.keras.layers.Activation("relu6",name="pre_act")(x)
    x = QuantSimul.QuantActivation(qunatization_start=ACT_QUANT_START, name="pre_act_quantized")(x)

    #blocks_1
    x = BasicBlock_QuantSimul(x, channel_sizes, "blocks_1_1", strides=1)
    for i in range(1,3):
        x = BasicBlock_QuantSimul(x, channel_sizes, "blocks_1_"+str(i+1), strides=1)

    #blocks_2
    x = BasicBlock_QuantSimul(x, channel_sizes*2, "blocks_2_1", strides=2)
    for i in range(1,3):
        x = BasicBlock_QuantSimul(x, channel_sizes*2, "blocks_2_"+str(i+1), strides=1)

    #blocks_3
    x = BasicBlock_QuantSimul(x, channel_sizes*4, "blocks_3_1", strides=2)
    for i in range(1,3):
        x = BasicBlock_QuantSimul(x, channel_sizes*4, "blocks_3_"+str(i+1), strides=1)
    
    #pred
    x = tf.keras.layers.GlobalAveragePooling2D(name="pred_gap")(x)
    x = QuantSimul.QuantSimulDense(classes, name="pred_dense", kernel_initializer=kernel_init, kernel_regularizer=kernel_reg)(x) #BIAS REGULARIZER X
    x = tf.keras.layers.Activation("softmax", name="pred_out")(x)
    outputs = QuantSimul.QuantActivation(qunatization_start=ACT_QUANT_START, name="pred_out_quantized")(x)
    
    return tf.keras.Model(inputs=inputs, outputs=outputs, name="ResNet20")

In [4]:
model_pretrained = tf.keras.models.load_model("BASELINE_ResNet20_CIFAR10_2.h5")
model_quantized = ResNet20_QuantSimul()

In [5]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

y_train = tf.keras.utils.to_categorical(y_train)# tf.squeeze( tf.one_hot( y_train, y_train.max()+1 ) )
y_test = tf.keras.utils.to_categorical(y_test) #tf.squeeze( tf.one_hot( y_test, y_test.max()+1 ) )

x_train = x_train/255.0
x_test = x_test/255.0
x_mean = np.mean(x_train, axis=(0,1,2), keepdims=True)
x_std = np.std(x_train, axis=(0,1,2), keepdims=True)

x_train = (x_train - x_mean)/x_std
x_test = (x_test - x_mean)/x_std

In [6]:
for layer in model_pretrained.layers:
    if type(layer) == tf.keras.layers.Conv2D:
        model_quantized.get_layer(layer.name).set_weights( layer.get_weights() )
    elif type(layer) == tf.keras.layers.Dense:
        model_quantized.get_layer(layer.name).set_weights( layer.get_weights() )
    elif type(layer) == tf.keras.layers.BatchNormalization:
        model_quantized.get_layer(layer.name).set_weights( layer.get_weights() )

In [7]:
def scheduler(epoch, lr):
    if epoch %30 == 0:
        return lr * 0.1
    else:
        return lr
    
LR_Scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [8]:
savename = "QuantSimul_ResNet20_CIFAR10"
checkpoint_filepath = './' + savename + '/checkpoint-{epoch}'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,save_weights_only=True,save_best_only=False)

In [9]:
optim = tf.keras.optimizers.SGD(1e-4*10, momentum=0.9)
loss_f = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
model_quantized.compile(optimizer=optim, loss=loss_f, metrics=['accuracy'])

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode='nearest',
    horizontal_flip=True)
datagen.fit(x_train)

#history = model_quantized.fit(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE), validation_data=(x_test, y_test), epochs=100, callbacks=[model_checkpoint_callback, LR_Scheduler])

Epoch 1/100
391/391 [==============================] - 54s 90ms/step - loss: 0.1485 - accuracy: 0.9514 - val_loss: 0.4285 - val_accuracy: 0.8839 - lr: 1.0000e-04
Epoch 2/100
391/391 [==============================] - 33s 84ms/step - loss: 0.1319 - accuracy: 0.9581 - val_loss: 0.4215 - val_accuracy: 0.8850 - lr: 1.0000e-04
Epoch 3/100
391/391 [==============================] - 34s 88ms/step - loss: 0.1207 - accuracy: 0.9617 - val_loss: 0.4415 - val_accuracy: 0.8742 - lr: 1.0000e-04
Epoch 4/100
391/391 [==============================] - 34s 87ms/step - loss: 0.1154 - accuracy: 0.9641 - val_loss: 0.4077 - val_accuracy: 0.8896 - lr: 1.0000e-04
Epoch 5/100
391/391 [==============================] - 34s 87ms/step - loss: 0.1074 - accuracy: 0.9675 - val_loss: 0.4842 - val_accuracy: 0.8803 - lr: 1.0000e-04
Epoch 6/100
391/391 [==============================] - 34s 86ms/step - loss: 0.1067 - accuracy: 0.9670 - val_loss: 0.4381 - val_accuracy: 0.8868 - lr: 1.0000e-04
Epoch 7/100
391/391 [=======

In [10]:
hist_df = pd.DataFrame(history.history)
hist_df.to_csv('QuantSimul_ResNet20_CIFAR10.csv'.format(savename), index=False)

#pd.read_csv("BASELINE_ResNet20_CIFAR10.csv")

In [13]:
model_quantized.evaluate(x_test, y_test)

313/313 [==============================] - 7s 17ms/step - loss: 0.6597 - accuracy: 0.8329


[0.6597059965133667, 0.8328999876976013]

In [9]:
optim = tf.keras.optimizers.SGD(1e-4*10, momentum=0.9)
loss_f = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
model_quantized.compile(optimizer=optim, loss=loss_f, metrics=['accuracy'])

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode='nearest',
    horizontal_flip=True)
datagen.fit(x_train)


model_to_test = tf.keras.models.load_model("QuantSimul_ResNet20_CIFAR10.h5")

In [12]:
model_to_test.evaluate(x_test, y_test)

313/313 [==============================] - 13s 22ms/step - loss: 0.4639 - accuracy: 0.9040


[0.4639362692832947, 0.9039999842643738]

In [ ]:
model_quantized.load_weights('./' + savename + '/checkpoint-87')

In [ ]:
model_quantized.save("QuantSimul_ResNet20_CIFAR10.h5")